### Generating a sample dataset

In [4]:
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib
import requests
import re

A PO number consists of PO + seven random digits + C, so below we'll define a function that will output a random PO number

In [5]:
def generate_po():
    result = str(random.sample(range(1000000,9999999), 1))
    result = result.strip('[')
    result = result.strip(']')
    return f"PO{result}C"

In [6]:
generate_po()

'PO1964861C'

The eventual idea is to locate PO numbers within the text of emails, so for the sample data set we want to define a loop that will use the above function to generate a random PO number, and then interpolate it into random text a set number of times, and then concatenate these outputs into a list. 

In [7]:
email_list = []
for i in itertools.repeat(None, 50):
    random_po = generate_po()
    sample_email = f'''Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt 
    ut labore et dolore magna aliqua. Magna {random_po} ac placerat vestibulum lectus mauris ultrices eros. 
    Duis tristique sollicitudin nibh sit amet. Odio eu feugiat pretium nibh ipsum consequat. Fusce ut placerat orci nulla 
    pellentesque dignissim enim sit amet.  Dictum {random_po} varius duis at consectetur. 
    Elementum pulvinar etiam non quam lacus suspendisse. Mi quis hendrerit dolor magna eget est lorem. 
    Pharetra convallis posuere morbi leo urna. Turpis egestas pretium aenean pharetra. 
    Amet cursus sit amet dictum sit amet justo donec.'''
    email_list.append(sample_email)


In [8]:
email_list[:3]

['Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt \n    ut labore et dolore magna aliqua. Magna PO7758576C ac placerat vestibulum lectus mauris ultrices eros. \n    Duis tristique sollicitudin nibh sit amet. Odio eu feugiat pretium nibh ipsum consequat. Fusce ut placerat orci nulla \n    pellentesque dignissim enim sit amet.  Dictum PO7758576C varius duis at consectetur. \n    Elementum pulvinar etiam non quam lacus suspendisse. Mi quis hendrerit dolor magna eget est lorem. \n    Pharetra convallis posuere morbi leo urna. Turpis egestas pretium aenean pharetra. \n    Amet cursus sit amet dictum sit amet justo donec.',
 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt \n    ut labore et dolore magna aliqua. Magna PO9287468C ac placerat vestibulum lectus mauris ultrices eros. \n    Duis tristique sollicitudin nibh sit amet. Odio eu feugiat pretium nibh ipsum consequat. Fusce ut placerat orci nulla \n    p

Because we'll eventually be searching the description column of the Salesforce report as a standard text file, we want to take the 50 'emails' in the above list and join them into one string which we'll then save as a text file to use later. 

In [9]:
sample_txt_file = ''.join(email_list)

In [10]:
sample_txt_file[:500]

'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt \n    ut labore et dolore magna aliqua. Magna PO7758576C ac placerat vestibulum lectus mauris ultrices eros. \n    Duis tristique sollicitudin nibh sit amet. Odio eu feugiat pretium nibh ipsum consequat. Fusce ut placerat orci nulla \n    pellentesque dignissim enim sit amet.  Dictum PO7758576C varius duis at consectetur. \n    Elementum pulvinar etiam non quam lacus suspendisse. Mi quis hendrerit dolor magna e'

In [11]:
text_file = open("sample_file.txt", "wt")
n = text_file.write(sample_txt_file)
text_file.close()

### Extracting the POs from emails using regular expressions

Now that we've generated a random dataset, we can move on to pulling the actual information. The first step will be to open the file we've created above as a string.

In [12]:
filepath = "sample_file.txt"
with open (filepath) as f:
    cases_str = f.read()

In [13]:
cases_str[:500]

'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt \n    ut labore et dolore magna aliqua. Magna PO7758576C ac placerat vestibulum lectus mauris ultrices eros. \n    Duis tristique sollicitudin nibh sit amet. Odio eu feugiat pretium nibh ipsum consequat. Fusce ut placerat orci nulla \n    pellentesque dignissim enim sit amet.  Dictum PO7758576C varius duis at consectetur. \n    Elementum pulvinar etiam non quam lacus suspendisse. Mi quis hendrerit dolor magna e'

Now that we've loaded the file we're going to search it for anything matching the PO pattern of 'PO + Seven Digits + C'

In [14]:
pattern = r"PO\d{7}C"
po_list = re.findall(pattern, cases_str)

In [15]:
po_list[:10]

['PO7758576C',
 'PO7758576C',
 'PO9287468C',
 'PO9287468C',
 'PO8978359C',
 'PO8978359C',
 'PO6173749C',
 'PO6173749C',
 'PO4567941C',
 'PO4567941C']

We've now got a list of every single time anything matching the PO pattern is mentioned in one of the sample 'emails' but you can see that some POs are mentioned more than once so we need to remove the duplicates.

In [16]:
po_series = pd.DataFrame(po_list)
po_series.rename(columns={0: "PO-Number"},inplace=True)
po_series.head()

,PO-Number
0,PO7758576C
1,PO7758576C
2,PO9287468C
3,PO9287468C
4,PO8978359C


In [17]:
po_series.drop_duplicates(inplace=True)

In [18]:
po_series.head(10)

,PO-Number
0,PO7758576C
2,PO9287468C
4,PO8978359C
6,PO6173749C
8,PO4567941C
10,PO1988874C
12,PO7537363C
14,PO2171100C
16,PO1594691C
18,PO1983557C


In [19]:
po_series.reset_index(inplace=True)
po_series.head()

,index,PO-Number
0,0,PO7758576C
1,2,PO9287468C
2,4,PO8978359C
3,6,PO6173749C
4,8,PO4567941C


In [20]:
po_series.drop(columns='index',inplace=True)
po_series.head()

,PO-Number
0,PO7758576C
1,PO9287468C
2,PO8978359C
3,PO6173749C
4,PO4567941C


In [21]:
po_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PO-Number  50 non-null     object
dtypes: object(1)
memory usage: 528.0+ bytes


After removing duplicates and cleaning up the table's labels and indexes, we can see that there were 50 unique PO numbers mentioned in the entirety of the emails. Now we can save these 50 unique PO numbers as a CSV, which we can then use to query the subcat of the PO. 

In [22]:
po_series.to_csv('po_list.csv')

In [23]:
example_dict = {'Swinton,': 1,
 'each': 1,
 'in': 4,
 'which': 2,
 'published': 1,
 'is': 6,
 '(Tilda': 1,
 'to': 1,
 'showcased': 1,
 'Ennui-sur-Blasé': 1,
 'perfect': 1,
 'own': 1,
 'as': 2,
 'magazine': 2,
 'tour': 1,
 'town,': 1,
 'Murray),': 1,
 'stories': 1,
 'scheduled': 1,
 'its': 3,
 'filmmakers': 1,
 'While': 1,
 'French': 2,
 'get': 1,
 'town': 1,
 'exteriors': 1,
 'multiple': 1,
 'list': 1,
 'Andersonian': 1,
 'movie’s': 1,
 'their': 1,
 'creating': 1,
 'theaters': 1,
 'found': 1,
 'a': 5,
 'that': 2,
 'Oct.': 1,
 'segments.': 1,
 'planning': 1,
 'film’s': 1,
 'Howitzer': 1,
 'heart,': 1,
 'movie,': 1,
 'dramatization': 1,
 'Frances': 1,
 '22,': 1,
 'editor,': 1,
 'of': 6,
 'longtime': 1,
 'backlot,': 1,
 'final': 1,
 'touch).': 1,
 'the': 10,
 'death': 1,
 'long': 1,
 'and': 2,
 'than': 1,
 'focuses': 1,
 'names': 1,
 'Del': 1,
 'an': 1,
 'Rather': 1,
 'makes': 1,
 'Angoulême,': 1,
 'on': 3,
 'Toro,': 1,
 'Benicio': 1,
 '(Bill': 1,
 'McDormand,': 1,
 'beating': 1,
 'Jr.': 1,
 'The': 1,
 'stars': 1,
 'used': 1,
 'or': 1,
 'With': 1,
 'Jeffrey': 1,
 'released': 1,
 'four': 1,
 'down': 1,
 'biggest': 1,
 'film.': 1,
 'fictional': 1,
 'issue,': 1,
 'Arthur': 1,
 'cast': 1,
 'recognizable': 1,
 '(the': 1,
 'name': 1,
 'one': 2,
 'city': 1,
 'dressing': 1,
 'needed.': 1,
 'city,': 1,
 'real': 1,
 'be': 1,
 'it': 2,
 'American': 1,
 'Wright)': 1,
 'up': 2}

In [24]:
pd.from_dict(example_dict)

AttributeError: module 'pandas' has no attribute 'from_dict'